## 환경설정

In [ ]:
# OPENAI_API
import os

os.environ['OPENAI_API_KEY'] = 'OPENAI API KEY 입력'

In [1]:
# 토큰 정보로드를 위한 라이브러리
# 설치: pip install python-dotenv
from dotenv import load_dotenv

# 토큰 정보로드
load_dotenv()

## 데이터 로드

pandas를 활용하여 csv 파일을 DataFrame 으로 로드합니다.

In [2]:
import pandas as pd

df = pd.read_csv('data/titanic.csv')
df.head()

,survived,pclass,sex,age,sibsp,parch,fare,embarked,class,who,adult_male,deck,embark_town,alive,alone
0,0,3,male,22.0,1,0,7.2500,S,Third,man,True,NaN,Southampton,no,False
1,1,1,female,38.0,1,0,71.2833,C,First,woman,False,C,Cherbourg,yes,False
2,1,3,female,26.0,0,0,7.9250,S,Third,woman,False,NaN,Southampton,yes,True
3,1,1,female,35.0,1,0,53.1000,S,First,woman,False,C,Southampton,yes,False
4,0,3,male,35.0,0,0,8.0500,S,Third,man,True,NaN,Southampton,no,True


## Pandas DataFrame Agent

In [17]:
from langchain.agents import create_pandas_dataframe_agent
from langchain.chat_models import ChatOpenAI
from langchain.agents.agent_types import AgentType


# 에이전트 생성
agent = create_pandas_dataframe_agent(
    ChatOpenAI(temperature=0, 
               model='gpt-4-0613'),        # 모델 정의
    df,                                    # 데이터프레임
    verbose=True,                          # 추론과정 출력
    agent_type=AgentType.OPENAI_FUNCTIONS, # AgentType.ZERO_SHOT_REACT_DESCRIPTION
)

In [18]:
# 질의
agent.run('데이터의 행과 열의 갯수는 어떻게 돼?')



> Entering new AgentExecutor chain...

Invoking: `python_repl_ast` with `{'query': 'df.shape'}`


(891, 15)데이터프레임 `df`는 891개의 행과 15개의 열로 구성되어 있습니다.

> Finished chain.


'데이터프레임 `df`는 891개의 행과 15개의 열로 구성되어 있습니다.'

In [12]:
# 질의
agent.run('남자 승객의 생존율을 어떻게 돼? %로 알려줘')



> Entering new AgentExecutor chain...
Thought: To find the survival rate of male passengers, I need to filter the dataframe for rows where 'sex' is 'male', then calculate the mean of the 'survived' column. This will give the proportion of male passengers who survived, which I can then multiply by 100 to get a percentage.
Action: python_repl_ast
Action Input: male_survival_rate = df[df['sex'] == 'male']['survived'].mean() * 100
Observation: 
Thought:Now I need to print the survival rate to see the result.
Action: python_repl_ast
Action Input: print(male_survival_rate)
Observation: 18.890814558058924

Thought:I now know the final answer
Final Answer: 남자 승객의 생존율은 약 18.89%입니다.

> Finished chain.


'남자 승객의 생존율은 약 18.89%입니다.'

In [19]:
# 질의
agent.run('나이가 15세 이하인 승객중 1,2등급에 탑승한 남자 승객의 생존율은 어떻게 돼? %로 알려줘')



> Entering new AgentExecutor chain...

Invoking: `python_repl_ast` with `{'query': "df_child_male = df[(df['age'] <= 15) & (df['sex'] == 'male') & (df['pclass'].isin([1, 2]))]\nchild_male_survival_rate = df_child_male['survived'].mean() * 100\nchild_male_survival_rate"}`


100.0나이가 15세 이하인 승객 중 1,2등급에 탑승한 남자 승객의 생존율은 100%입니다.

> Finished chain.


'나이가 15세 이하인 승객 중 1,2등급에 탑승한 남자 승객의 생존율은 100%입니다.'

## 2개 이상의 DataFrame

2개 이상의 데이터프레임에 기반한 LLM 기반 질의를 할 수 있습니다. 2개 이상의 데이터프레임 입력시 `[]` 로 묶어주면 됩니다.

In [20]:
# 샘플 데이터프레임 생성
df1 = df.copy()
df1 = df1.fillna(0)
df1.head()

,survived,pclass,sex,age,sibsp,parch,fare,embarked,class,who,adult_male,deck,embark_town,alive,alone
0,0,3,male,22.0,1,0,7.2500,S,Third,man,True,0,Southampton,no,False
1,1,1,female,38.0,1,0,71.2833,C,First,woman,False,C,Cherbourg,yes,False
2,1,3,female,26.0,0,0,7.9250,S,Third,woman,False,0,Southampton,yes,True
3,1,1,female,35.0,1,0,53.1000,S,First,woman,False,C,Southampton,yes,False
4,0,3,male,35.0,0,0,8.0500,S,Third,man,True,0,Southampton,no,True


In [23]:
# 에이전트 생성
agent = create_pandas_dataframe_agent(
    ChatOpenAI(temperature=0, 
               model='gpt-4-0613'),
               [df, df1], 
               verbose=True
)

# 질의
agent.run('나이 컬럼의 나이의 평균차이는 어떻게 돼? %로 구해줘.')



> Entering new AgentExecutor chain...
Thought: To find the average age difference between the two dataframes, I need to calculate the average age in each dataframe and then find the difference. I will then convert this difference to a percentage.

Action: python_repl_ast
Action Input: df1['age'].mean()
Observation: 29.69911764705882
Thought:The average age in df1 is approximately 29.7. Now I will calculate the average age in df2.

Action: python_repl_ast
Action Input: df2['age'].mean()
Observation: 23.79929292929293
Thought:The average age in df2 is approximately 23.8. Now I will calculate the difference between the two averages.

Action: python_repl_ast
Action Input: abs(df1['age'].mean() - df2['age'].mean())
Observation: 5.899824717765892
Thought:The difference in average age between df1 and df2 is approximately 5.9. Now I will convert this difference to a percentage of the average age in df1.

Action: python_repl_ast
Action Input: (abs(df1['age'].mean() - df2['age'].mean()) / df1[

'The average age difference between the two dataframes is approximately 19.87%.'